In [1]:
from __future__ import print_function
import requests
import httplib2
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
import requests
import json
import os
from requests.auth import HTTPBasicAuth
import gspread
from oauth2client.service_account import ServiceAccountCredentials

In [2]:
try:
    API_ENDPOINT = "http://sisense3.nationsinfocorp.com:8081/api/v1/authentication/login" 
    payload = {'username':'BI','password':'goNat10ns'}
    session = requests.Session()
    response = session.post(url = API_ENDPOINT, data = payload, auth=HTTPBasicAuth('sisense','b2Sense#2'))
    response.raise_for_status() #throw exception if request does not retun 2xx
    # print(response.text)
    access_token = json.loads(response.text)['access_token']
    headers = {"PassThruAuth": "Bearer " + access_token}
    # print(headers)
except requests.exceptions.RequestException as e: 
    print (e)
# Dash API Call
try:
    url = "http://sisense3.nationsinfocorp.com:8081/api/v1/dashboards"
    response = requests.get(url, headers=headers,auth=HTTPBasicAuth('sisense','b2Sense#2'))
    response.raise_for_status() #throw exception if request does not return 2xx
    # print(response)
    # print(response.text)
    dashes = json.loads(response.text)
except requests.exceptions.RequestException as e: 
    print (e)

In [3]:
# Dash Definition
def getsubset(data):
    # change to a list
    l = []
    for i in data:
        l.append(i)
    return l


def visit(data, i):
    # get the ith item in data
    return(data[getsubset(data)[i]])


def getwidget(data):
    # get the widgets of a dash
    return(data["widgets"])


def getPanel(widget):
    # get the panel of a widget(rows, values, columns, filters)
    return(widget["metadata"]["panels"])


def panel(widget, string):
    # search the subpanel of a widget by subpanel name(rows, values, columns, filters)
    t = "not found"
    for i in range(len(getPanel(widget))):
        if getPanel(widget)[i]["name"] == string:
            t = getPanel(widget)[i]
            break
    return([t, i])


def getitem(panel):
    # get the items of a panel
    return(panel["items"])


def getjaql(item):
    return(item["jaql"])


def jaql(panel, string):
    # search JAQL by title(name)
    t = "not found"
    item = getitem(panel)
    for i in range(len(item)):
        if getjaql(item[i])["title"] == string:
            t = getjaql(item[i])
            break
    return([t, i])


def measure(context):
    t = "t"
    if "dim" in context:
        t = "Bottom"
    elif "type" in context:
        if context["type"] == "measure":
            t = "Measure"
    elif len(context) > 0 and "type" not in context:
        t = "Collection"
    return(t)


def explorer(context, l=[], string="", count=0):

    if measure(context) == "Bottom":
        l.append(string+"|"+str(count)+"." +
                 context["table"]+"|"+context["title"]+"|"+context["column"])
    if measure(context) == "Measure":
        count = count+1
        explorer(context["context"], l, string+"|" +
                 str(count)+"."+context["title"], count)
    if measure(context) == "Collection":
        for i in context:
            explorer(context[i], l, string, count)
    return(l)


def metricMap(context, l=[], string="", count=0, l2=[], main="", cube=""):
    # 'Metric name', 'cube Name', 'table Name', 'metric title in formula', 'column name']]
    if measure(context) == "Bottom":
        if count == 1:
            c = [main, cube, context["table"],
                 context["title"], context["column"]]
        elif count == 0:
            c = [context["dim"], cube, context["table"],
                 context["title"], context["column"]]
        else:
            c = [string, cube, context["table"],
                 context["title"], context["column"]]
        if c not in l:
            l.append(c)
    if measure(context) == "Measure":
        count = count+1
        if count == 1:
            mainm = context["title"]
            c1 = [cube, mainm, mainm]
            if c1 not in l2:
                l2.append(c1)
            metricMap(context["context"], l, mainm, count, l2, mainm, cube)
        else:
            c2 = [cube, main, context["title"]]
            c3 = [cube, string, context["title"]]
            if c2 not in l2:
                l2.append(c2)
            if c3 not in l2:
                l2.append(c3)
            metricMap(context["context"], l,
                      context["title"], count, l2, main, cube)
    if measure(context) == "Collection":
        for i in context:
            metricMap(context[i], l, string, count, l2, main, cube)
    return(l)

In [4]:
# Get values from Sisense to Write to Spreadsheet
values = [['Folder', 'Dashboard', 'Widget',
           'Field_Type', 'DashID', 'Metrics', 'cube']]
metrics = [['cube', 'Metric', 'Subset']]
location = [['Metric', 'cube', 'table', 'title', 'column']]

for l in range(len(dashes)):  
    try:
        oid = dashes[l]["oid"]
        daily = json.loads(requests.get(url+"/"+oid+"/export/dash",
                                        headers=headers, auth=HTTPBasicAuth('sisense', 'b2Sense#2')).text)
        dname = daily["title"]
        cube = daily["datasource"]["title"]
        url1 = "http://sisense3.nationsinfocorp.com:8081/api/v1/folders/" + \
            daily['parentFolder']
        folder = json.loads(requests.get(url1, headers=headers, auth=HTTPBasicAuth('sisense', 'b2Sense#2')).text)
        if "parentId" in getsubset(folder):
            url2 = "http://sisense3.nationsinfocorp.com:8081/api/v1/folders/" + \
                daily['parentFolder']+"/ancestors"
            foldername = json.loads(requests.get(url2, headers=headers, auth=HTTPBasicAuth(
                'sisense', 'b2Sense#2')).text)[0]["name"]
        else:
            foldername = folder["name"]
    except:
        continue

    for k in range(len(getwidget(daily))):
        try:
            widget = getwidget(daily)[k]
            wname = widget["title"]

            for j in range(len(getPanel(widget))):
                item = getitem(getPanel(widget)[j])
                name = getPanel(widget)[j]["name"]
                for i in range(len(getsubset(item))):
                    row = [foldername, dname, wname, name, daily["oid"], getjaql(item[i])[
                        "title"], cube]
                    values.append(row)
                    e = metricMap(
                        getjaql(item[i]), location, "", 0, metrics, "", cube)
        except:
            continue

In [ ]:
### Write to Google Sheet

In [34]:
os.chdir(r'C:\Users\Yu Zhang\Desktop\Project')
# Scope for Google Api
scope = ["https://spreadsheets.google.com/feeds",'https://www.googleapis.com/auth/spreadsheets',"https://www.googleapis.com/auth/drive.file","https://www.googleapis.com/auth/drive"]
# creds = ServiceAccountCredentials.from_json_keyfile_name("creds.json", scope)
creds = ServiceAccountCredentials.from_json_keyfile_name("creds_bi_dash_documentation.json", scope)
client = gspread.authorize(creds)

In [37]:
# # # Create a Spreadsheet and share it with your email 
# sh = client.create('Dash Documentation')   
# sh.share('bi_team@nationsinfocorp.com', perm_type='user', role='writer')

In [38]:
sheet = client.open("Dash Documentation")  # Open the spreadhseet

In [39]:
# Create a Worksheet
worksheet1 = sheet.add_worksheet(title="Dash", rows="1000", cols="8")
worksheet2 = sheet.add_worksheet(title="Metric", rows="1000", cols="5")
worksheet3 = sheet.add_worksheet(title="Table",rows="1000", cols="8")

In [40]:
# Write to different worksheets in google sheet
sheet.values_update(
    'Dash!A1', 
    params={'valueInputOption': 'RAW'}, 
    body={'values': values}
)

{'spreadsheetId': '1K_arC_0xh0EAnCxthVDYKPGQOgr4CkgeoNYV1A1laM8',
 'updatedRange': 'Dash!A1:G69218',
 'updatedRows': 69218,
 'updatedColumns': 7,
 'updatedCells': 484526}

In [41]:
sheet.values_update(
    'Metric!A1', 
    params={'valueInputOption': 'RAW'}, 
    body={'values': metrics}
)

{'spreadsheetId': '1K_arC_0xh0EAnCxthVDYKPGQOgr4CkgeoNYV1A1laM8',
 'updatedRange': 'Metric!A1:C9272',
 'updatedRows': 9272,
 'updatedColumns': 3,
 'updatedCells': 27816}

In [42]:
sheet.values_update(
    'Table!A1', 
    params={'valueInputOption': 'RAW'}, 
    body={'values': location}
)

{'spreadsheetId': '1K_arC_0xh0EAnCxthVDYKPGQOgr4CkgeoNYV1A1laM8',
 'updatedRange': 'Table!A1:E18802',
 'updatedRows': 18802,
 'updatedColumns': 5,
 'updatedCells': 94010}